# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 24 2023 2:42PM,258146,10,PRF-54065,Bio-PRF,Released
1,Feb 24 2023 2:42PM,258146,10,PRF-54076,Bio-PRF,Released
2,Feb 24 2023 2:42PM,258146,10,PRF-54080,Bio-PRF,Released
3,Feb 24 2023 2:37PM,258145,10,37744,"Digital Brands, LLC",Released
4,Feb 24 2023 2:37PM,258145,10,37745,"Digital Brands, LLC",Released
5,Feb 24 2023 2:37PM,258145,10,37746,"Digital Brands, LLC",Released
6,Feb 24 2023 2:37PM,258145,10,37747,"Digital Brands, LLC",Released
7,Feb 24 2023 2:37PM,258145,10,37748,"Digital Brands, LLC",Released
8,Feb 24 2023 2:34PM,258144,10,MSP218877,Methapharm-G,Released
9,Feb 24 2023 2:34PM,258144,10,MSP218890,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,258134,Released,1
17,258140,Released,1
18,258144,Released,2
19,258145,Released,5
20,258146,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258134,NaN,NaN,1.0
258140,NaN,NaN,1.0
258144,NaN,NaN,2.0
258145,NaN,NaN,5.0
258146,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258089,0.0,0.0,5.0
258090,0.0,0.0,56.0
258094,0.0,0.0,1.0
258099,13.0,6.0,4.0
258109,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258089,0,0,5
258090,0,0,56
258094,0,0,1
258099,13,6,4
258109,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258089,0,0,5
1,258090,0,0,56
2,258094,0,0,1
3,258099,13,6,4
4,258109,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258089,,,5
1,258090,,,56
2,258094,,,1
3,258099,13,6,4
4,258109,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 24 2023 2:42PM,258146,10,Bio-PRF
3,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC"
8,Feb 24 2023 2:34PM,258144,10,Methapharm-G
10,Feb 24 2023 1:02PM,258140,10,Bio-PRF
11,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC"
12,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation
25,Feb 24 2023 11:28AM,258130,10,ISDIN Corporation
26,Feb 24 2023 11:28AM,258129,10,"Snap Medical Industries, LLC"
28,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation
30,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 24 2023 2:42PM,258146,10,Bio-PRF,,,3
1,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC",,,5
2,Feb 24 2023 2:34PM,258144,10,Methapharm-G,,,2
3,Feb 24 2023 1:02PM,258140,10,Bio-PRF,,,1
4,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",,,1
5,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,,,13
6,Feb 24 2023 11:28AM,258130,10,ISDIN Corporation,,,1
7,Feb 24 2023 11:28AM,258129,10,"Snap Medical Industries, LLC",,,2
8,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,,,2
9,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 2:42PM,258146,10,Bio-PRF,3,,
1,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC",5,,
2,Feb 24 2023 2:34PM,258144,10,Methapharm-G,2,,
3,Feb 24 2023 1:02PM,258140,10,Bio-PRF,1,,
4,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",1,,
5,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,13,,
6,Feb 24 2023 11:28AM,258130,10,ISDIN Corporation,1,,
7,Feb 24 2023 11:28AM,258129,10,"Snap Medical Industries, LLC",2,,
8,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,2,,
9,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 2:42PM,258146,10,Bio-PRF,3,,
1,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC",5,,
2,Feb 24 2023 2:34PM,258144,10,Methapharm-G,2,,
3,Feb 24 2023 1:02PM,258140,10,Bio-PRF,1,,
4,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 2:42PM,258146,10,Bio-PRF,3,NaN,NaN
1,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC",5,NaN,NaN
2,Feb 24 2023 2:34PM,258144,10,Methapharm-G,2,NaN,NaN
3,Feb 24 2023 1:02PM,258140,10,Bio-PRF,1,NaN,NaN
4,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 2:42PM,258146,10,Bio-PRF,3,0.0,0.0
1,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC",5,0.0,0.0
2,Feb 24 2023 2:34PM,258144,10,Methapharm-G,2,0.0,0.0
3,Feb 24 2023 1:02PM,258140,10,Bio-PRF,1,0.0,0.0
4,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3871874,124,4.0,2.0
19,516193,5,6.0,13.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3871874,124,4.0,2.0
1,19,516193,5,6.0,13.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,124,4.0,2.0
1,19,5,6.0,13.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,124.0
1,19,Released,5.0
2,10,Executing,4.0
3,19,Executing,6.0
4,10,Completed,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19
Status,,
Completed,2.0,13.0
Executing,4.0,6.0
Released,124.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19
0,Completed,2.0,13.0
1,Executing,4.0,6.0
2,Released,124.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19
0,Completed,2.0,13.0
1,Executing,4.0,6.0
2,Released,124.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()